# Bitcoin Key Pairs

In [1]:
# pip install ecdsa

## Manual Key Generation

In [2]:
import ecdsa
import binascii
import hashlib


# # Generate a private key
private_key_raw_object = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print("Private key raw:", (private_key_raw_object.to_string().hex()) + '\x01')

private_key_raw = private_key_raw_object.to_string()

# Get the private key
# private_key_hex = private_key_raw.to_string().hex()
# private_key_bytes = binascii.unhexlify(private_key_hex)
private_key = ecdsa.SigningKey.from_string(private_key_raw, curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print("Private key:", private_key.to_string().hex())

# Get the public key
# public_key: ecdsa.VerifyingKey = private_key.get_verifying_key()
public_key = private_key.get_verifying_key()

# Print the public key in hexadecimal format
print(f"Public key compressed hex: {public_key.to_string("compressed").hex()}")
print(f"Public key uncompressed hex: {public_key.to_string("uncompressed").hex()}")


Private key raw: e2b5f6538effcf0080503e4acb1e0ec11ae1632844883d47a0f936595b767038
Private key: e2b5f6538effcf0080503e4acb1e0ec11ae1632844883d47a0f936595b767038
Public key compressed hex: 028d4455ba8d8dcfc29c1c40a3d354b6d0fcd8fe0e90922377da14ccf06ddc86b2
Public key uncompressed hex: 048d4455ba8d8dcfc29c1c40a3d354b6d0fcd8fe0e90922377da14ccf06ddc86b2a0dfd1b1c05a060a9dbd1213527cb87c713be45e89e592209bb41e5507b352ec


While the public key is correct it is not in the proper format. 

## Bitcoin Address

Finding the bitcoin public key or bitcoin address from the private key is a bit more complicated.

The public key is a point on the elliptic curve, but it is not in the proper format. The proper format is a 65 byte array with the first byte being 0x04 and the next 32 bytes being the x coordinate and the last 32 bytes being the y coordinate. The x and y coordinates are 32 bytes each because they are 256 bit numbers. The first byte is 0x04 because it is an uncompressed public key. There are compressed public keys which are 33 bytes long, but we will not be using them.

### Create a bitcoin address

In [3]:
import hashlib
import base58

# Generate a bitcoin address
# Step 1: Hash public key
sha256: bytes = hashlib.sha256(public_key.to_string('compressed')).digest()
print("SHA256: ", sha256.hex())

# Step 2: Perform RIPEMD-160 hashing on the result of SHA256
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(sha256)
hashed_public_key = ripemd160.digest()
print("RIPEMD-160: ", hashed_public_key.hex())

# Step 3: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
hashed_public_key = b'\x00' + hashed_public_key

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(hashed_public_key).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]
print("Address checksum: ", address_checksum.hex())

# Step 6: Add the 4 checksum bytes from stage 5 at the end of extended RIPEMD-160 hash from stage 4. This is the 25-byte binary Bitcoin Address.
binary_bitcoin_address = hashed_public_key + address_checksum
print("Binary Bitcoin Address: ", binary_bitcoin_address.hex())

# Step 7: Convert the result from a byte string into a base58 string using Base58Check encoding. This is the most commonly used Bitcoin Address format
bitcoin_address = base58.b58encode(binary_bitcoin_address).decode()
print("Bitcoin address: ", bitcoin_address)

SHA256:  2e1243d7b0309d4f0052c2ecd13a4b072c3ae75e48cfcb44c33a700ee19818d1
RIPEMD-160:  52d92c99eeff040fd973836fdeaa29b51fbbd5d1
Address checksum:  091d90ba
Binary Bitcoin Address:  0052d92c99eeff040fd973836fdeaa29b51fbbd5d1091d90ba
Bitcoin address:  18Z4aG4eNhwvjTVn6fK7Lrbz9CaULMznvV


### Verify a bitcoin address

In [4]:
# Verify that the bitcoin address is correct
# Step 1: Decode Base58 Bitcoin Address
binary_bitcoin_address_check = base58.b58decode(bitcoin_address)

# Step 2: Extract Bitcoin Address from decoded Base58 Bitcoin Address
binary_bitcoin_address = binary_bitcoin_address_check[:-4]

# Step 3: Extract Bitcoin Address Checksum from decoded Base58 Bitcoin Address
binary_bitcoin_address_checksum = binary_bitcoin_address_check[-4:]

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(binary_bitcoin_address).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Verify that the address checksum from decoded Base58 Bitcoin Address is equal to the address checksum from the SHA256 hash
print("Bitcoin address is valid:", binary_bitcoin_address_checksum == address_checksum)

Bitcoin address is valid: True


## Signing Messages

### Sign a message

In [5]:
def encode_varint(i: int) -> bytes:
    """
    Encode a potentially very large integer into varint bytes. The length should be
    specified in little-endian.

    https://bitcoin.org/en/developer-reference#compactsize-unsigned-integers
    """
    if i < 253:
        return bytes([i])
    elif i < 0x10000:
        return b"\xfd" + i.to_bytes(2, "little")
    elif i < 0x100000000:
        return b"\xfe" + i.to_bytes(4, "little")
    elif i < 0x10000000000000000:
        return b"\xff" + i.to_bytes(8, "little")
    else:
        raise ValueError("Integer is too large: %d" % i)

In [6]:
# Path: concepts_bitcoin_transactions.ipynb
import base64

private_key_raw_1 = private_key_raw_object
msg = b"Hello world!"
msg_size = encode_varint(len(msg))
message_hash = hashlib.sha256(hashlib.sha256(b'\x18Bitcoin Signed Message:\n' + msg_size + msg).digest()).digest()
print("Message hash:", message_hash.hex())

signature_ = private_key_raw_1.sign_deterministic(message_hash)
# Use the private key to sign a message
# message = bytes("Bitcoin Signed Message:\nHello world!", "utf-8")
# message = b"\x18Bitcoin Signed Message:\n" + msg_size + msg
# message_size = encode_varint(len(message))
# signature_ = private_key.sign_deterministic(hashlib.sha256(hashlib.sha256(message).digest()).digest())
# print("Signature:", signature_.hex())
print(f'Signature in b64: {base64.b64encode(bytes([28]) + signature_).decode('ascii')}')
print("Signature hex:", binascii.hexlify(signature_).decode('ascii'))

# Verify the signature
print(public_key.verify(signature_, message_hash))


Message hash: 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
Signature in b64: HAPIdfaLSOAdHqTZgyfKO0YjhUPjHfaEKmzT1EO/llk2xEkHRRqwPeKBDFmrvA2z1t3/Wh5fPV5MWrcA4pYw+Uw=
Signature hex: 03c875f68b48e01d1ea4d98327ca3b46238543e31df6842a6cd3d443bf965936c44907451ab03de2810c59abbc0db3d6ddff5a1e5f3d5e4c5ab700e29630f94c
True


In [7]:
# Get the private key
# private_key_hex = 'your_private_key_here'
# private_key_bytes = bytes.fromhex(private_key_hex)
# private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)

# The message to sign
# message = 'your_message_here'

# Sign the message
signature = private_key_raw_1.sign(message_hash)
print(f"Signature: {signature.hex()}")

signature_b64 = base64.b64encode(bytes([28]) + signature).decode('ascii')
print(f"Signature b64: {signature_b64}")

# Print the signature in base64 format
# signature_b64 = base64.b64encode(signature).decode('ascii')
# signature_b64 = base64.b64decode(signature)
# print(f"Signature b64: {signature_b64}")

# Get the public key
public_key: ecdsa.VerifyingKey = private_key_raw_1.get_verifying_key()
print(f"Public key: {public_key.to_string('compressed').hex()}")


Signature: 66e320150507b7e8150cd3ecccdaba69664dbe230b63ad882345dc3154eba336872c4de6ebfb6b427d84069eeec3f44f7e27241f53e8b84686b23d807c904840
Signature b64: HGbjIBUFB7foFQzT7MzaumlmTb4jC2OtiCNF3DFU66M2hyxN5uv7a0J9hAae7sP0T34nJB9T6LhGhrI9gHyQSEA=
Public key: 028d4455ba8d8dcfc29c1c40a3d354b6d0fcd8fe0e90922377da14ccf06ddc86b2


### Verify a message

In [8]:
# Use the public key to verify a message
# Verify the signature
try:
    public_key.verify(signature, message_hash)
    print("Signature valid")
except ecdsa.BadSignatureError:
    print("Signature invalid")

Signature valid


# Libraries for bitcoin keys

### pybitcoin

No longer maintained and does not work with python 3.12

In [9]:
# pip install pybitcoin

In [10]:
# from pybitcoin import BitcoinPrivateKey

# # Generate a private key
# private_key = BitcoinPrivateKey()
# print(f"Private key: {private_key.to_hex()}")

# # Generate a public key
# public_key = private_key.public_key()
# print(f"Public key: {public_key.to_hex()}")

# # Generate a bitcoin address
# bitcoin_address = public_key.address()
# print(f"Bitcoin address: {bitcoin_address}")

### bitcoin-utils

Package appears damaged and does not work with python 3.12

In [11]:
# pip install bitcoin-utils

In [12]:
# from bitcoinutils.keys import PrivateKey, PublicKey

# # Generate a private key
# private_key = PrivateKey()
# print(f"Private key: {private_key.to_wif(compressed=True)}")

# # get the public key
# public_key: PublicKey = private_key.get_public_key()
# print(f"Public key: {public_key.to_hex(compressed=True)}")

# # get the bitcoin address
# bitcoin_address = public_key.get_address()
# print(f"Bitcoin address: {bitcoin_address.to_string()}")
# print(f'Hash160: {bitcoin_address.to_hash160()}')



Sign a message with a private key and verify the signature with the public key.

In [13]:
# message: str = "Hello"
# signature = private_key.sign_message(message)
# print(f"Signature: {signature}")
# # print(f"Signature valid? {public_key.verify(message, signature)}")
# print(f"Signature valid? {public_key.verify_message(public_key.get_address().to_string(), message, signature)}")

### python-bitcoinlib

In [14]:
# pip install python-bitcoinlib

In [15]:
import secrets
import base64
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress, CBitcoinAddress
from bitcoin.signmessage import BitcoinMessage, VerifyMessage, SignMessage
from bitcoin.core import x, lx, b2x, b2lx, Hash160

message_: str = "Hello world!"

# Generate a private key
private_key_ = CBitcoinSecret.from_secret_bytes(secrets.token_bytes(32))
print(f"Private key: {private_key_}")

# Generate a public key
public_key_ = private_key_.pub
print(f"Public key: {public_key_.hex()}")

# Generate a bitcoin address
bitcoin_address_ = P2PKHBitcoinAddress.from_pubkey(public_key_)
# bitcoin_address_ = CBitcoinAddress(bitcoin_address_)
print(f"Bitcoin address: {bitcoin_address_}")

# Sign message
bitcoin_msg = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_msg.magic} {bitcoin_msg.message} {bitcoin_msg.GetHash().hex()}")

signature_ = SignMessage(private_key_, bitcoin_msg)
print(f"Signature: {signature_.hex()}")
print(f'signature in b64: {base64.b64decode(bytes([32]) + signature_).hex()}')

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_, bitcoin_msg, signature_)}")


Private key: Kx8mFgzxTKQnnU8zHBt7mohuwDhNmSCcbhMJ1bHqawfAWjzWbPks
Public key: 0266bce94eabc9eda9a407623673861415152e781083be19ea8fb517f20b5f07fb
Bitcoin address: 1GGHR5iyMky2UobpDxm6C6PDJYD1MHMAdZ
Bitcoin message: b'Bitcoin Signed Message:\n' b'Hello world!' 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
Signature: 49446b574d6947664d34396775427a742b5a4d51497836437639663278465842536e63544f34644b7376416a626e6474512b33495a76554f336b58544e7572584e4d466e4b7261384677546438387a496e6b7149364f513d
signature in b64: 20391632219f338f60b81cedf99310231e82bfd7f6c455c14a77133b874ab2f0236e776d43edc866f50ede45d336ead734c1672ab6bc1704ddf3ccc89e4a88e8e4
Signature valid? True


In [16]:
# verify the handmade message from the top
# print(f'Private Key BAse58 Encoded: {base58.b58encode(private_key_raw + b"\x01")}')
print(f'Private key not encoded: {private_key_raw.hex() + "01"}')
# bitccoin_private_key = CBitcoinSecret.from_secret_bytes(private_key_raw)
bitccoin_private_key = CBitcoinSecret.from_secret_bytes(bytes.fromhex(private_key_raw.hex() + "01"), 0)
bitccoin_private_key_bytes = bitccoin_private_key

# print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()[:-2]}")
print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()}")

# bitcoin_public_key_bytes = bitccoin_private_key_bytes.pub
# print(f'Public key base58 Encoded: {base58.b58encode(private_key.get_verifying_key().to_string('compressed'))}')
print(f'bitcoin public not encoded: {private_key.get_verifying_key().to_string('compressed').hex()}')

def compress_public_key(public_key: bytes) -> bytes:
    x = public_key[:32]
    y = public_key[32:]
    if int.from_bytes(y, 'big') % 2 == 0:
        return b'\x02' + x
    else:
        return b'\x03' + x
    
# bitcoin_public_key_bytes = compress_public_key(private_key_raw.get_verifying_key().to_string())
bitcoin_public_key_bytes = private_key.get_verifying_key().to_string('compressed')
print(f"Bitcoin public key: {bitcoin_public_key_bytes.hex()}")

print(f"Bitcoin address from top: {bitcoin_address}")
bitcoin_address_bytes = P2PKHBitcoinAddress.from_pubkey(bytes.fromhex(bitcoin_public_key_bytes.hex()))
print(f"Bitcoin address from pubkey: {bitcoin_address_bytes}")

# bitcoin_address_bytes = P2PKHBitcoinAddress.from_bytes(bytes.fromhex(bitcoin_public_key_bytes.hex()), 0)
# print(f"Bitcoin address: {bitcoin_address_bytes}")


Private key not encoded: e2b5f6538effcf0080503e4acb1e0ec11ae1632844883d47a0f936595b76703801
Bitcoin private key: e2b5f6538effcf0080503e4acb1e0ec11ae1632844883d47a0f936595b76703801
bitcoin public not encoded: 028d4455ba8d8dcfc29c1c40a3d354b6d0fcd8fe0e90922377da14ccf06ddc86b2
Bitcoin public key: 028d4455ba8d8dcfc29c1c40a3d354b6d0fcd8fe0e90922377da14ccf06ddc86b2
Bitcoin address from top: 18Z4aG4eNhwvjTVn6fK7Lrbz9CaULMznvV
Bitcoin address from pubkey: 18Z4aG4eNhwvjTVn6fK7Lrbz9CaULMznvV


In [17]:

bitcoin_message_bytes = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_message_bytes.magic} {bitcoin_message_bytes.message} {bitcoin_message_bytes.GetHash().hex()}")

print(f'signature from top: {signature_.decode()}')
# print(f'signature from top b64: {base64.b64encode(signature)}')
# signature_bytes = signature.__bytes__()
signature_bytes_b64 = base64.b64encode(bytes([32]) + signature_).hex()
print(f"Signature from top b64: {signature_bytes_b64}")

#sign message using bitcoin library
signature_bytes = SignMessage(bitccoin_private_key, bitcoin_message_bytes)
print(f"Signature bytes: {signature_bytes.decode()}")

# print(f'signature_bytes_hex: {base64.b64encode(signature_bytes).hex()}')
print(f'signature bytes decoded: {base64.b64decode(signature_bytes).hex()}')

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_bytes, bitcoin_message_bytes, signature_bytes)}")

Bitcoin message: b'Bitcoin Signed Message:\n' b'Hello world!' 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
signature from top: IDkWMiGfM49guBzt+ZMQIx6Cv9f2xFXBSncTO4dKsvAjbndtQ+3IZvUO3kXTNurXNMFnKra8FwTd88zInkqI6OQ=
Signature from top b64: 49456c456131644e6155646d545451355a335643656e5172576b315253586732513359355a6a4a34526c68435532356a564538305a45747a646b4671596d356b644645724d306c61646c56504d32745956453531636c684f54555a7553334a684f455a33564751344f48704a626d747853545a505554303d
Signature bytes: H2e1RXIuSiKmRK2y3bojl0kGr6WLSC+DVRLSw1KymJBxT6bCMm0nRV6n/nyyp59GrxB+/dEvr3wRezIWS+YYTHk=
signature bytes decoded: 1f67b545722e4a22a644adb2ddba23974906afa58b482f835512d2c352b29890714fa6c2326d27455ea7fe7cb2a79f46af107efdd12faf7c117b32164be6184c79
Signature valid? True


In [18]:
# Get the private key
private_key_hex = private_key_raw.hex()
private_key_bytes = binascii.unhexlify(private_key_hex)
private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)
print(f"Private key: {private_key.to_string().hex()}")

# Get the public key
public_key_hex = private_key.get_verifying_key().to_string('compressed').hex()
public_key_bytes = binascii.unhexlify(public_key_hex)
public_key = ecdsa.VerifyingKey.from_string(public_key_bytes, curve=ecdsa.SECP256k1)
print(f"Public key: {public_key.to_string('compressed').hex()}")

# Get the bitcoin address
bitcoin_address = P2PKHBitcoinAddress.from_pubkey(public_key_bytes)
print(f"Bitcoin address: {bitcoin_address}")

# The message to sign
message = bitcoin_message_bytes.GetHash()
print(f"Message: {message.hex()}")

# Sign the message
signature = private_key.sign(message)

# Print the signature in hexadecimal format
print(f"Signature: {binascii.hexlify(signature).decode()}")
print(f"Signature ascii: {binascii.hexlify(signature).decode('ascii')}")

signature_hex = binascii.hexlify(signature).decode('ascii')

# The signature
signature_unhex = binascii.unhexlify(signature_hex.encode('ascii'))
print(f"Signature unhex: {signature_unhex}")

# signature from b64
# signature_from_b64 = base64.b64decode(signature_hex)
# print(f"Signature from b64: {signature_from_b64}")

# signature base64
signature_b64 = base64.b64encode(bytes([32]) + signature_unhex).decode('ascii')
print(f"Signature b64: {signature_b64}")

# Verify the signature
try:
    public_key.verify(signature_unhex, message)
    print("Signature valid")
except ecdsa.BadSignatureError:
    print("Signature invalid")

Private key: e2b5f6538effcf0080503e4acb1e0ec11ae1632844883d47a0f936595b767038
Public key: 028d4455ba8d8dcfc29c1c40a3d354b6d0fcd8fe0e90922377da14ccf06ddc86b2
Bitcoin address: 18Z4aG4eNhwvjTVn6fK7Lrbz9CaULMznvV
Message: 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
Signature: 536d3184ebc76a4c5c1a1c957f17ddc8e084d86c705285df9ceeeef0bf079842626e7227c49412435d9e41ae0dcb64f456832bed445153400e8b96b48c9d1774
Signature ascii: 536d3184ebc76a4c5c1a1c957f17ddc8e084d86c705285df9ceeeef0bf079842626e7227c49412435d9e41ae0dcb64f456832bed445153400e8b96b48c9d1774
Signature unhex: b"Sm1\x84\xeb\xc7jL\\\x1a\x1c\x95\x7f\x17\xdd\xc8\xe0\x84\xd8lpR\x85\xdf\x9c\xee\xee\xf0\xbf\x07\x98Bbnr'\xc4\x94\x12C]\x9eA\xae\r\xcbd\xf4V\x83+\xedDQS@\x0e\x8b\x96\xb4\x8c\x9d\x17t"
Signature b64: IFNtMYTrx2pMXBoclX8X3cjghNhscFKF35zu7vC/B5hCYm5yJ8SUEkNdnkGuDctk9FaDK+1EUVNADouWtIydF3Q=
Signature valid


In [19]:
# pip install pybitcoin

In [20]:
# !pip install bitcoin

# from pybitcoin import privtopub, ecdsa_raw_sign, encode_sig

# # Get the private key and message
# private_key = 'your_private_key_here'
# message = 'your_message_here'

# # Get the public key
# public_key = privtopub(private_key)

# # Sign the message
# r, s = ecdsa_raw_sign(message, private_key)

# # Calculate the recovery id
# recovery_id = 27 + (public_key[64] % 2)

# # Encode the signature in compact format
# compact_signature = encode_sig(recovery_id, r, s)

# print(f"Compact signature: {compact_signature}")

In [21]:
import hashlib
import ecdsa
import binascii
from bitcoin.signmessage import BitcoinMessage, VerifyMessage, SignMessage

def sign_message(private_key_hex, message):
    private_key_bytes = binascii.unhexlify(private_key_hex)
    private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)

    magic_message = b'\x18Bitcoin Signed Message:\n' + len(message).to_bytes(1, 'big') + message.encode()
    magic_message_hash = hashlib.sha256(hashlib.sha256(magic_message).digest()).digest()

    signature = private_key.sign(magic_message_hash)
    print(f"Signature: {signature}")
    # return binascii.hexlify(signature).decode()
    return signature

private_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print(f"Private key: {private_key.to_string()}")

private_key_hex = private_key.to_string().hex()
print(f"Private key hex: {private_key_hex}")

message = 'Hello world!'
print(f"Message: {message}")

signature = sign_message(private_key_hex, message)
print(f"Signature hex: {signature.hex()}")

signature_b64 = base64.b64encode(bytes([32]) + signature).decode()
print(f"Signature b64: {signature_b64}")

# Verify the signature
def verify_signature(public_key_hex, signature, message):
    public_key_bytes = binascii.unhexlify(public_key_hex)
    public_key = ecdsa.VerifyingKey.from_string(public_key_bytes, curve=ecdsa.SECP256k1)

    magic_message = b'\x18Bitcoin Signed Message:\n' + len(message).to_bytes(1, 'big') + message.encode()
    magic_message_hash = hashlib.sha256(hashlib.sha256(magic_message).digest()).digest()

    return public_key.verify(signature, magic_message_hash)

public_key = private_key.get_verifying_key()
public_key_formatted = '04' + public_key.to_string().hex()
print(f"Public key formatted: {'04' + public_key.to_string().hex()}")

public_key_hex = public_key.to_string('compressed').hex()
print(f"Public key compressed hex: {public_key_hex}")

public_key_uncompressed = public_key.to_string('uncompressed').hex()
print(f"Public key uncompressed: {public_key_uncompressed}")

print(f"Signature valid (Check from pubkey)? {verify_signature(public_key_hex, signature, message)}")

# verify the signature using the python-bitcoinlib library
bitcoin_message = BitcoinMessage(message)
print(f"Bitcoin message: {bitcoin_message.magic} {bitcoin_message.message} {bitcoin_message.GetHash().hex()}")

bitcoin_private_key = CBitcoinSecret.from_secret_bytes(bytes.fromhex(private_key_hex), 0)
print(f"Bitcoin private key: {bitcoin_private_key}")
print(f"Bitcoin private key Hex: {bitcoin_private_key.hex()}")

# bitcoin_public_from_bitcoin_private = bitcoin_private_key.pub

bitcoin_public_key = P2PKHBitcoinAddress.from_pubkey(bytes.fromhex(public_key_hex))
print(f"Bitcoin public key: {bitcoin_public_key}")
print(f'bitcoin public key hex: {bitcoin_public_key.to_bytes().hex()}')
# print(f"Bitcoin public key Hex: {bitcoin_public_key.to_scriptPubKey().hex()}")

bitcoin_public_key = bitcoin_private_key.pub
print(f"Bitcoin public key: {bitcoin_public_key}")
print(f"Bitcoin public key hex: {bitcoin_public_key.hex()}")

bitcoin_address = P2PKHBitcoinAddress.from_pubkey(bitcoin_public_key)
print(f"Bitcoin address: {bitcoin_address}")

bitcoin_signed_message = SignMessage(bitcoin_private_key, bitcoin_message)
print(f"Bitcoin signed message: {bitcoin_signed_message}")
print(f"Bitcoin signed message base64decode: {base64.b64decode(bitcoin_signed_message).hex()}")

print(f"Signature valid? {VerifyMessage(bitcoin_address, bitcoin_message, bitcoin_signed_message)}")


Private key: b'\x8dK\xfd[.\xb2J\xae\xe2Z\x04/p)\x98\xf1U@3\xe9\xf9\xf0\xb0\xe5C~\x05\xd0F\xb2Bx'
Private key hex: 8d4bfd5b2eb24aaee25a042f702998f1554033e9f9f0b0e5437e05d046b24278
Message: Hello world!
Signature: b'\x8b~H\xee\xef\xdf2YNI#\xce\xde&\xc8\x85\xa1\xd6H]\xea]\\\xafx\xd9;G\xa7\xa6\xb6\xd1\xfd\xee\xd0\xa6\xec(B\xfc\xb66\xe2\xebl\x0b\x80m\x04\x8e\xd3\x15@\x00\x13\xcdiM\xfd\x86Pt{\x14'
Signature hex: 8b7e48eeefdf32594e4923cede26c885a1d6485dea5d5caf78d93b47a7a6b6d1fdeed0a6ec2842fcb636e2eb6c0b806d048ed315400013cd694dfd8650747b14
Signature b64: IIt+SO7v3zJZTkkjzt4myIWh1khd6l1cr3jZO0enprbR/e7QpuwoQvy2NuLrbAuAbQSO0xVAABPNaU39hlB0exQ=
Public key formatted: 0498ca5e5bfb0c0b570c604446882746cb7fdadfd9adf1f812c48cfd47f30f72bf7650d7ba0caf2f223815684a8770599b20a06a4f14531b1ecc3d11a4ba8e263f
Public key compressed hex: 0398ca5e5bfb0c0b570c604446882746cb7fdadfd9adf1f812c48cfd47f30f72bf
Public key uncompressed: 0498ca5e5bfb0c0b570c604446882746cb7fdadfd9adf1f812c48cfd47f30f72bf7650d7ba0caf2f22381